In [1]:
import heapq
import itertools
import math
import os
from typing import Iterable, Callable

import numpy as np
from scipy.stats import mannwhitneyu

import utils
from Core.PRef import PRef
from Core.PS import PS
from Gian_experimental.NSGAIICustom.NSGAIICustom import NSGAIICustom, NCSolution, NCSamplerSimple, NCMutationSimple, \
    NCCrossoverSimple, EvaluatedNCSolution, NCSamplerFromPRef, NCCrossoverTransition, NCMutation, \
    NCMutationCounterproductive
from PolishSystem.OperatorsBasedOnSimilarities.similarities_utils import get_transition_matrix
from PolishSystem.OperatorsBasedOnSimilarities.similarities_utils import gian_get_similarities
from PolishSystem.read_data import get_pRef_from_vectors


def make_similarity_atomicity(similarities):
    def atomicity(ps):
        if len(ps) < 2:
            return -1000
        else:
            linkages = [similarities[a, b] for a, b in itertools.combinations(ps, r=2)]
            return np.average(linkages)

    return atomicity


def get_matches_non_matches_in_pRef(ps, given_pRef: PRef):
    ps_true_values = np.full(shape=250, fill_value=-1, dtype=int)
    ps_true_values[list(ps)] = 1
    return given_pRef.fitnesses_of_observations_and_complement(PS(ps_true_values))


def make_consistency_metric_with_sample_size(pRef: PRef,
                                             threshold: float = 0.5,
                                             must_match_at_least: int = 3):
    def consistency_and_sample(ps):
        # matches, non_matches = sPRef.get_matching_fitnesses_and_not_matching(ps, threshold=threshold)
        matches, non_matches = get_matches_non_matches_in_pRef(ps, pRef)
        if min(len(matches), len(non_matches)) < must_match_at_least:
            return 1, len(matches)
        else:
            test = mannwhitneyu(matches, non_matches, alternative="greater", method="asymptotic")
            return test.pvalue, len(matches)
            # return permutation_mannwhitney_u(matches, non_matches, n_permutations=50), len(matches)

    return consistency_and_sample


def make_min_metric_with_sample_size(pRef: PRef):
    def min_and_sample(ps):
        matches, non_matches = get_matches_non_matches_in_pRef(ps, pRef)
        if min(len(matches), len(non_matches)) < 1:
            return (-1000, len(matches))
        else:
            lowest_fitness = np.min(matches)
        return lowest_fitness, len(matches)

    return min_and_sample


class HashedSolution:
    solution: NCSolution

    def __init__(self,
                 sol):
        self.solution = sol

    def __hash__(self):
        return sum(self.solution) % 7787

    def __eq__(self, other):
        return self.solution == other.solution


def make_metrics_cached(metrics):
    cached_values = dict()

    def get_values(ps):
        wrapped = HashedSolution(ps)
        if wrapped in cached_values:
            return cached_values[wrapped]
        else:

            value = metrics(ps)
            cached_values[wrapped] = value
            return value

    return get_values




In [2]:
from typing import Iterable

from Gian_experimental.NSGAIICustom.NSGAIICustom import NSGAIICustom, NCSolution, EvaluatedNCSolution


# we want to show how the operators affect the sample_size of the children as the program progresses

def run_NSGAII_in_steps(algorithm: NSGAIICustom) -> Iterable[Iterable[EvaluatedNCSolution]]:
    # this will yield every generation

    used_evaluations = [0]

    def with_fitnesses(solution: NCSolution) -> EvaluatedNCSolution:
        fitnesses = algorithm.mo_fitness_function(solution)
        used_evaluations[0] += 1
        return EvaluatedNCSolution(solution, fitnesses)
    def sampler_yielder():
        while True:
            yield with_fitnesses(algorithm.sampling.sample())

    algorithm.log("Beginning of NC process")
    population = algorithm.make_population(yielder=sampler_yielder(), required_quantity=algorithm.pop_size)
    yield population

    while (used_evaluations[0] < algorithm.eval_budget):
        population = algorithm.make_next_generation(population, with_fitnesses)
        yield population
        algorithm.log(f"Used evals: {used_evaluations[0]}")
    pareto_fronts = algorithm.get_pareto_fronts(population)

    if algorithm.unique:
        return list(set(pareto_fronts[0]))
    return pareto_fronts[0]




In [3]:

dir_250 = r"C:\Users\gac8\PycharmProjects\PSSearch\data\retail_forecasting\250"


def in_250(path):
    return os.path.join(dir_250, path)


def count_frequencies(iterable):
    iterable_list = list(iterable)
    counts = {item: iterable_list.count(item)
              for item in set(iterable)}

    for key, count in counts.items():
        print(key, count)



In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def compare_distributions_histogram(
    datasets,
    bin_count=50,
    xlim=None,
    labels=None,
    title="Histogram Comparison",
    density=False,
    alpha=0.8,
    colors=None,
    logx=False,
    logy=False,
    fig_size = (10, 6)
):
    """
    Compare multiple distributions using side-by-side histograms.

    Parameters:
    - datasets: List of lists or arrays of numbers
    - bin_count: Number of bins to use (even for log scale)
    - xlim: Tuple (xmin, xmax) for x-axis limits; if None, auto-calculated
    - labels: List of labels for each dataset
    - title: Title of the plot
    - density: If True, plot probability densities instead of counts
    - alpha: Transparency of the histogram bars
    - colors: List of colors for the histograms
    - logx: If True, set x-axis to log scale (requires positive data)
    - logy: If True, set y-axis to log scale
    """
    datasets = [np.asarray(data) for data in datasets]
    
    if logx:
        datasets = [data[data > 0] for data in datasets]

    # Determine x-axis limits
    if xlim is None:
        xmin = min(data.min() for data in datasets)
        xmax = max(data.max() for data in datasets)
        xlim = (xmin, xmax)

    # Define bins
    if logx:
        bins = np.logspace(np.log10(xlim[0]), np.log10(xlim[1]), bin_count + 1)
    else:
        bins = np.linspace(xlim[0], xlim[1], bin_count + 1)

    # Histogram counts
    hists = [np.histogram(data, bins=bins, density=density)[0] for data in datasets]
    bin_centers = (bins[:-1] + bins[1:]) / 2
    width = np.diff(bins)

    num_datasets = len(datasets)
    offsets = np.linspace(-0.4, 0.4, num=num_datasets) * width[:, None]

    if labels is None:
        labels = [f"Data {i+1}" for i in range(num_datasets)]

    if colors is None:
        colors = [f"C{i}" for i in range(num_datasets)]

    # Plotting
    plt.figure(figsize=fig_size)
    for i, hist in enumerate(hists):
        center_shift = offsets[:, i]
        plt.bar(bin_centers + center_shift, hist,
                width=width * (0.8 / num_datasets),
                label=labels[i],
                alpha=alpha,
                color=colors[i % len(colors)],
                edgecolor='black',
                align='center')

    plt.xlabel("Value (log scale)" if logx else "Value")
    plt.ylabel("Density" if density else "Count")
    plt.title(title)

    # Axis scaling
    ax = plt.gca()
    if logx:
        ax.set_xscale("log")
        ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=15))
        ax.xaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * 0.1,
                                                     numticks=100))
        ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    else:
        ax.set_xscale("linear")

    if logy:
        ax.set_yscale("log")

    plt.xlim(xlim)
    plt.legend()
    plt.grid(True, which='both' if logx or logy else 'major', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [5]:



with utils.announce("Loading the pRef and other data"):
    pRef = get_pRef_from_vectors(name_of_vectors_file=in_250("many_hot_vectors_250_random.csv"),
                                 name_of_fitness_file=in_250("fitness_250_random.csv"),
                                 column_in_fitness_file=2)

    train_pRef, test_pRef = pRef.train_test_split(test_size=0.3)

    print(f"{train_pRef.sample_size = }, {test_pRef.sample_size = }")
    cluster_info_file_name = in_250(f"cluster_info_250_qmc.pkl")
    similarities = gian_get_similarities(cluster_info_file_name)
    n = pRef.full_solution_matrix.shape[1]





starting [Loading the pRef and other data]
[Loading the pRef and other data]...Finished (took 0.188740 seconds)


ValueError: the number of columns changed from 250 to 64 at row 4646; use `usecols` to select a subset and avoid this error

In [ ]:


with utils.announce("Printing the stats for each variable"):
    print(f"The pRef has {pRef.sample_size}, where the variables appear")
    quantity_by_variable = [(index, np.sum(pRef.full_solution_matrix[:, index]))
                            for index in range(n)]
    top_10_most_common = heapq.nlargest(n = 10, iterable=quantity_by_variable, key=utils.second)
    print("The top 10 most common vars are: "+", ".join(f"{var =}:{samples = }" for var, samples in top_10_most_common))


In [ ]:

with utils.announce("Making the operators and metrics"):
    custom_sampling = NCSamplerFromPRef.from_PRef(train_pRef)
    transition_matrix = get_transition_matrix(similarities)
    custom_crossover = NCCrossoverTransition(transition_matrix)
    custom_mutation = NCMutationCounterproductive(transition_matrix, disappearance_probability=0.9)

    # train_mean_fitness = make_mean_fitness(train_pRef, threshold=threshold)
    train_atomicity = make_similarity_atomicity(similarities)
    train_consistency_and_sample = make_consistency_metric_with_sample_size(train_pRef, must_match_at_least=3)
    train_min_and_sample = make_min_metric_with_sample_size(train_pRef)
    test_consistency_and_sample = make_consistency_metric_with_sample_size(test_pRef, must_match_at_least=3)



    traditional_sampling = NCSamplerSimple.with_average_quantity(3, genome_size=n)
    traditional_mutation = NCMutationSimple(n)

    traditional_crossover = NCCrossoverSimple(swap_probability=1 / n)


In [ ]:

with utils.announce("Constructing the algorithm"):
    def get_metrics(ps: NCSolution) -> tuple[float]:
        p_value, sample_size = train_consistency_and_sample(ps)
        atomicity = train_atomicity(ps)
        return (-sample_size, p_value, -atomicity)

    def keep_ones_with_most_samples(population: Iterable[EvaluatedNCSolution], quantity_required: int):
        return heapq.nsmallest(iterable=population, key=lambda x: x.fitnesses[0], n=quantity_required)

    baseline_algorithm = NSGAIICustom(sampling=traditional_sampling,
                             mutation=traditional_mutation,
                             crossover=traditional_crossover,
                             probability_of_crossover=0.5,
                             eval_budget=1000,
                             pop_size=50,
                             tournament_size=3,
                             mo_fitness_function=make_metrics_cached(get_metrics),
                             unique=True,
                             verbose=True,
                             culler=keep_ones_with_most_samples)


In [ ]:
generations = list(run_NSGAII_in_steps(baseline_algorithm))


        

In [ ]:
def sensible_intervals(data, intervals):
    sorted_data = sorted(data)
    indices_to_consider  = []
    last_index = 0
    while last_index < len(data):
        indices_to_consider.append(math.floor(last_index))
        last_index += len(data) / (intervals + 1)
    
    indices_to_consider.append(len(data)-1)
    
    to_show = [(index, sorted_data[index]) for index in indices_to_consider]
    return to_show 
    
    


In [ ]:

for generation_index, generation in enumerate(generations):
    metric_arrays = np.array([ps.fitnesses for ps in generation])
    where_non_empty = np.array([len(ps.solution) > 0 for ps in generation])
    metric_arrays = metric_arrays[where_non_empty]
    
    # column 0, sample size
    data = -metric_arrays[:, 2]
    data = data[data < train_pRef.sample_size]
    intervals = (sensible_intervals(data, intervals=5))
    print("\t".join(f"{value:>6.3}" for value in intervals))
    

        